In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.optim as optim
from torcheval.metrics.functional import multiclass_f1_score

import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import copy
from tqdm import tqdm
import time
import os
from scipy import signal
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from natsort import natsorted

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [5]:
# Vamos definir a pasta que está o dataset
workspace = 'C:\\Meu Drive\\Doutorado Unicamp\\Projeto\\Dataset\\'
root = 'C:\\Meu Drive\\Doutorado Unicamp\\Projeto\\Dataset\\RealWorld\\'

# Lista de usuários e atividades
users = natsorted(os.listdir(root))
tarefas = ['climbingdown', 'climbingup', 'jumping', 'lying', 'running', 'sitting', 'standing', 'walking']
SAC = ['sitting', 'standing', 'walking', 'climbingup', 'climbingdown', 'running']
posicao = ['chest', 'forearm', 'head', 'shin', 'thigh', 'upperarm', 'waist']
posicaopt = ['peito', 'antebraço', 'cabeça', 'canela', 'coxa', 'braço', 'cintura']

In [6]:
pasta = 'C:\\Meu Drive\\Doutorado Unicamp\\Projeto\\Dataset\\realworld views\\'
D = [None] * 7
D[0] = pd.read_csv(pasta+posicao[0]+'_nova.csv')
D[1] = pd.read_csv(pasta+posicao[1]+'_nova.csv')
D[2] = pd.read_csv(pasta+posicao[2]+'_nova.csv')
D[3] = pd.read_csv(pasta+posicao[3]+'_nova.csv')
D[4] = pd.read_csv(pasta+posicao[4]+'_nova.csv')
D[5] = pd.read_csv(pasta+posicao[5]+'_nova.csv')
D[6] = pd.read_csv(pasta+posicao[6]+'_nova.csv')

In [11]:
D[0].shape

(22547, 363)

In [12]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()

        self.all_layers = torch.nn.Sequential(

            # Encoder
            nn.Flatten(start_dim=1),
            nn.Linear(360, 20),
            nn.ReLU(),

            # Decoder
            nn.Linear(20, 360),
            # nn.Softmax(dim=1),

#             # output layer
#             nn.Linear(15, num_classes),
        )

    def forward(self, x):
        logits = self.all_layers(x)
        return logits

In [19]:
X = D[0].values[:,:360]
y = D[0].values[:,-1]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1, stratify=y)
X_train = torch.tensor(X_train, dtype=torch.float32, device=device).detach()

In [20]:
model = MLP().to(device)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
n_epochs = 10   # number of epochs to run
batch_size = 250  # size of each batch
batch_start = torch.arange(0, len(X_train), batch_size)
trainLoss = []
valLoss = []

In [ ]:
# AJUSTAR ESSE TREINAMENTO!!
for epoch in range(n_epochs):
    model.train()
    with tqdm(batch_start, unit="batch", mininterval=0, disable=False) as bar:
        bar.set_description(f"Epoch {epoch}")
        for start in bar:
            # take a batch
            X_batch = X_train[start:start+batch_size,:,:]
            # forward pass
            Xh = model(X_batch)
            loss = loss_fn(Xh, X_batch)
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            # update weights
            optimizer.step()
            # print progress
            bar.set_postfix(loss=float(loss))
    # evaluate accuracy at end of each epoch
    model.eval()
    # y_pred = regress(encoder(X_val))
    y_pred = model(X_val)
    valLoss.append(float(loss_fn(y_pred, s_val)))
    # y_pred = regress(encoder(X_train))
    # trainLoss.append(float(loss_fn(y_pred, s_train)))

fig = go.Figure()
fig.add_trace(go.Scatter(y=valLoss, mode="lines", showlegend=True, name='validação'))
# fig.add_trace(go.Scatter(y=trainLoss, mode="lines", showlegend=True, name='treino'))
fig.show()